In [ ]:
!pip install dlib

     |████████████████████████████████| 3.2MB 10.7MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/96/ac/11/8aadec62cb4fb5b264a9b1b042caf415de9a75f5e165d79a51
Successfully built dlib
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
from pathlib import Path
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import shutil
import time
import dlib
from random import shuffle

In [ ]:
detector = dlib.cnn_face_detection_model_v1("/home/refinex/face-forensics/videos/mmod_human_face_detector.dat")

In [ ]:
source_path = '/home/refinex/DFDC_big/clustered_split/'
for split in ['train/','val/']:
    for label in ['real/','fake/']:
        print(source_path+split+label+': '+str(len([f for f in os.listdir(source_path+split+label) if 'mp4' in f])))

/home/refinex/DFDC_big/clustered_split/train/real/: 15107
/home/refinex/DFDC_big/clustered_split/train/fake/: 78591
/home/refinex/DFDC_big/clustered_split/val/real/: 4047
/home/refinex/DFDC_big/clustered_split/val/fake/: 21401


In [ ]:
source_path = '/home/refinex/DFDC_big/dlib_clustered_30faces/'
for split in ['train/','val/']:
    for label in ['real/','fake/']:
        Path(source_path+split+label).mkdir(parents=True, exist_ok=True)

In [ ]:
source_path = '/home/refinex/DFDC_big/clustered_split/'
dest_path = '/home/refinex/DFDC_big/dlib_clustered_30faces/'
split = 'train/'
label = 'real/'
vidpath = source_path + split + label
folderpath = dest_path + split + label
videos = sorted(os.listdir(source_path + split + label))
shuffle(videos)
videos = videos[:len([f for f in os.listdir(source_path+split+'real/') if 'mp4' in f])]
req_frames = 30
scale = 1.3
req_size = 200

In [ ]:
logs_dict = {}

In [ ]:
with open('trainreal_vids.txt','r') as f:
    videos = f.read().split('\n')

In [ ]:
for video in tqdm(videos[:3000]):
    facepath = folderpath + video[:-4] + '/'
    Path(facepath).mkdir(parents=True,exist_ok=True)
    video_capture = cv2.VideoCapture(vidpath + video)
    count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    more_than_one = 0
    no_faces = 0
    out_of_frame = 0
    done_frames = 0
    img_size = -1
    frame_number = 0
    while True:
        res, frame = video_capture.read()
        if not res:
            break
        faceRects = detector(frame[:,:,::-1], 0)
        if (len(faceRects)==0):
            no_faces += 1
            continue
        if (len(faceRects)>1):
            more_than_one += 1
            continue
        x1 = faceRects[0].rect.left()
        y1 = faceRects[0].rect.top()
        x2 = faceRects[0].rect.right()
        y2 = faceRects[0].rect.bottom()
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        length = max(x2 - x1, y2 - y1)
        x1 = center[0] - int(length*scale/2)
        x2 = center[0] + int(length*scale/2)
        y1 = center[1] - int(length*scale/2)
        y2 = center[1] + int(length*scale/2)
        if x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
            out_of_frame += 1
            continue
        final=frame[y1:y2,x1:x2,::-1]
        plt.imsave(facepath+str(frame_number).zfill(2)+'.jpg',final)
        frame_number += 1
        done_frames += 1
        if done_frames == req_frames:
            img_size = y2-y1
            break
    logs_dict[video] = [more_than_one, no_faces, out_of_frame, img_size, done_frames]
    video_capture.release()

100%|██████████| 3000/3000 [8:29:47<00:00,  4.73s/it]   


In [ ]:
import json

In [ ]:
with open('dlib_30faces_{}{}0_logs.json'.format(split[:-1],label[:-1]), 'w') as fp:
    json.dump(logs_dict, fp)